In [66]:
import pandas as pd

1. Load and merge separate Missouri and Kansas data files.
We have to adjust for dissimilarities between the two data sets, including the years and forms in which the data are offered (count vs. percent).

In [67]:
# load data
lunchMO = pd.read_excel('lunchMO.xlsx')
lunchKS = pd.read_excel('lunchKS.xlsx')

In [68]:
lunchMO.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4872 entries, 0 to 4871
Data columns (total 4 columns):
Location      4872 non-null object
TimeFrame     4872 non-null int64
DataFormat    4872 non-null object
Data          4872 non-null float64
dtypes: float64(1), int64(1), object(2)
memory usage: 152.3+ KB


In [82]:
# add column with state name
lunchMO['State'] = 'Missouri'

# remove raw numbers, because KS data only has percents
lunchMO = lunchMO[lunchMO.DataFormat.str.contains("Number") == False]

# remove years 1996-2002, because KS data starts at 2003
lunchMO = lunchMO[lunchMO['TimeFrame'] >= 2003]

# mark as part of KC or not, for future analysis purposes
# used official list of counties in Kansas City Metropolitan Statistical Area: 
# http://www.marc.org/Data-Economy/Metrodataline/General-Information/Statistical-Areas
lunchMO['KC_or_no'] = 'No'
lunchMO.loc[lunchMO['Location'].isin(['Cass','Clay','Clinton','Jackson','Lafayette',
                                 'Platte','Ray','Bates','Caldwell']), 'KC_or_no'] = 'Yes'

lunchMO.head()

,Location,TimeFrame,DataFormat,Data,State,KC_or_no
28,Missouri,2003,Percent,0.39202,Missouri,No
29,Missouri,2004,Percent,0.40508,Missouri,No
30,Missouri,2005,Percent,0.41700,Missouri,No
31,Missouri,2006,Percent,0.40731,Missouri,No
32,Missouri,2007,Percent,0.41727,Missouri,No


In [83]:
# add column with state name
lunchKS['State'] = 'Kansas'

# remove weird subgroups, like 'Frontier Counties'
lunchKS = lunchKS[lunchKS.Location.str.contains("Counties") == False]

# remove 2017, because MO data only goes to 2016
lunchKS = lunchKS[lunchKS['TimeFrame'] <= 2016]

# mark as part of KC or not, for future analysis purposes
lunchKS['KC_or_no'] = 'No'
lunchKS.loc[lunchKS['Location'].isin(['Johnson','Leavenworth','Miami','Wyandotte',
                                  'Franklin','Linn']), 'KC_or_no'] = 'Yes'

lunchKS.head()

,Location,TimeFrame,DataFormat,Data,State,KC_or_no
0,Kansas,2003,Percent,0.3604,Kansas,No
1,Kansas,2004,Percent,0.3751,Kansas,No
2,Kansas,2005,Percent,0.3858,Kansas,No
3,Kansas,2006,Percent,0.3887,Kansas,No
4,Kansas,2007,Percent,0.3901,Kansas,No


In [84]:
lunch = pd.concat([lunchMO,lunchKS])
lunch = lunch.reset_index()
lunch.head()

,index,Location,TimeFrame,DataFormat,Data,State,KC_or_no
0,28,Missouri,2003,Percent,0.39202,Missouri,No
1,29,Missouri,2004,Percent,0.40508,Missouri,No
2,30,Missouri,2005,Percent,0.41700,Missouri,No
3,31,Missouri,2006,Percent,0.40731,Missouri,No
4,32,Missouri,2007,Percent,0.41727,Missouri,No


In [85]:
lunch.tail()

,index,Location,TimeFrame,DataFormat,Data,State,KC_or_no
3103,1555,Washington,2016,Percent,0.43137,Kansas,No
3104,1556,Wichita,2016,Percent,0.58128,Kansas,No
3105,1557,Wilson,2016,Percent,0.56289,Kansas,No
3106,1558,Woodson,2016,Percent,0.63596,Kansas,No
3107,1559,Wyandotte,2016,Percent,0.74163,Kansas,Yes


Questions:
1. How did KC counties compare to statewide rate in 2016?
2. Where did KC counties rank among all counties in 2016?
3. How does change in KC counties (2003-2016) compare to statewide change?
4. How does change in KC counties (2003-2016) compare to change in all other counties?

In [88]:
# How did KC counties compare to statewide rate in 2016?
KC = lunch[(lunch.DataFormat == 'Percent') & 
           (lunch.KC_or_no == 'Yes') & (lunch.TimeFrame == 2016)]
KC.sort_values('Data', ascending=False)

,index,Location,TimeFrame,DataFormat,Data,State,KC_or_no
3107,1559,Wyandotte,2016,Percent,0.74163,Kansas,Yes
685,3149,Jackson,2016,Percent,0.60244,Missouri,Yes
3056,1508,Linn,2016,Percent,0.56813,Kansas,Yes
111,593,Bates,2016,Percent,0.54562,Missouri,Yes
2310,707,Franklin,2016,Percent,0.54430,Kansas,Yes
195,719,Caldwell,2016,Percent,0.51961,Missouri,Yes
769,3275,Lafayette,2016,Percent,0.46320,Missouri,Yes
363,1532,Clinton,2016,Percent,0.42217,Missouri,Yes
3063,1515,Miami,2016,Percent,0.41084,Kansas,Yes
1259,4010,Ray,2016,Percent,0.38736,Missouri,Yes


In [90]:
state = lunch[(lunch.DataFormat == 'Percent') & 
           (lunch['Location'].isin(['Missouri','Kansas'])) & (lunch.TimeFrame == 2016)]
state.sort_values('Data', ascending=False)

,index,Location,TimeFrame,DataFormat,Data,State,KC_or_no
13,41,Missouri,2016,Percent,0.51503,Missouri,No
1637,13,Kansas,2016,Percent,0.49273,Kansas,No


Result: There's a lot of variation in KC counties, but 6 of 15 exceeded both state averages in 2016.

In [138]:
# Where did KC counties rank among all counties in 2016?
counties = lunch[(lunch.DataFormat == 'Percent') & (lunch.TimeFrame == 2016)]
pd.options.display.height = 225
counties.sort_values('Data', ascending=False).reset_index()

height has been deprecated.



,level_0,index,Location,TimeFrame,DataFormat,Data,State,KC_or_no
0,1553,4766,Washington,2016,Percent,0.95508,Missouri,No
1,1623,4871,St. Louis City,2016,Percent,0.94615,Missouri,No
2,1105,3779,Pemiscot,2016,Percent,0.93459,Missouri,No
3,1427,4577,Shannon,2016,Percent,0.88028,Missouri,No
4,503,1742,Dunklin,2016,Percent,0.83348,Missouri,No
5,293,1427,Cedar,2016,Percent,0.81173,Missouri,No
6,951,3548,Mississippi,2016,Percent,0.81139,Missouri,No
7,3090,1542,Seward,2016,Percent,0.79997,Kansas,No
8,2309,706,Ford,2016,Percent,0.79087,Kansas,No
9,475,1700,Dent,2016,Percent,0.78943,Missouri,No


Result: Wyandotte ranks highest of the KC counties, at 14 among the 222 counties in both states. 
    Of the major cities, St. Louis is harder hit, where nearly 95 percent of students are enrolled. 
    But that might partially be due to differences in how the counties are structured: St. Louis County is segregated
    from the city's suburbs, while Kansas City's counties span urban and suburban areas.

In [143]:
# How does change (in percentage points) in KC counties (2003-2016) compare to statewide change?
KC0316 = lunch[(lunch.DataFormat == 'Percent') & (lunch.KC_or_no == 'Yes') & 
               (lunch['TimeFrame'].isin([2003,2016]))][['Location','Data']].groupby('Location')

In [ ]:
# all values increased, so we can just subtract max from min within each group
for item, county in KC0316:
    diff = (item, KC0316.Data.max() - KC0316.Data.min())

In [122]:
list(diff)

['Wyandotte', Location
 Bates          0.13762
 Caldwell       0.08439
 Cass           0.14379
 Clay           0.14357
 Clinton        0.14709
 Franklin       0.23060
 Jackson        0.16277
 Johnson        0.15402
 Lafayette      0.14886
 Leavenworth    0.13897
 Linn           0.18263
 Miami          0.14874
 Platte         0.12052
 Ray            0.11827
 Wyandotte      0.10513
 Name: Data, dtype: float64]

In [127]:
state0316 = lunch[(lunch.DataFormat == 'Percent') & (lunch['Location'].isin(['Missouri','Kansas'])) & 
               (lunch['TimeFrame'].isin([2003,2016]))][['Location','Data']].groupby('Location')

# both values increased, so we can just subtract max from min within each group
for item, state in state0316:
    diff2 = (item, state0316.Data.max() - state0316.Data.min())

list(diff2)

['Missouri', Location
 Kansas      0.13233
 Missouri    0.12301
 Name: Data, dtype: float64]

In [144]:
list(state0316)

[('Kansas',      Location     Data
  1624   Kansas  0.36040
  1637   Kansas  0.49273), ('Missouri',     Location     Data
  0   Missouri  0.39202
  13  Missouri  0.51503)]

Results: 11 of 15 counties increased by more percentage points than the states as a whole.

In [140]:
# How does change (in percentage points) in KC counties (2003-2016) compare to change in all other counties?
all0316 = lunch[(lunch.DataFormat == 'Percent') & (lunch.KC_or_no == 'No') & 
               (lunch['TimeFrame'].isin([2003,2016]))][['Location','Data']].groupby('Location')

# all values increased, so we can just subtract max from min within each group
for item, county in all0316:
    diff3 = (item, all0316.Data.max() - all0316.Data.min())
    
diff3

('Wright', Location
 Adair             0.17085
 Allen             0.17326
 Anderson          0.06586
 Andrew            0.07049
 Atchison          0.24869
 Audrain           0.28199
 Barber            0.11257
 Barry             0.14275
 Barton            0.22185
 Benton            0.26239
 Bollinger         0.13399
 Boone             0.12629
 Bourbon           0.15086
 Brown             0.16372
 Buchanan          0.10887
 Butler            0.43249
 Callaway          0.11154
 Camden            0.16518
 Cape Girardeau    0.18799
 Carroll           0.04188
 Carter            0.03318
 Cedar             0.37343
 Chariton          0.09917
 Chase             0.02449
 Chautauqua        0.11474
 Cherokee          0.14444
 Cheyenne          0.04616
 Christian         0.11659
 Clark             0.11624
 Clay              0.07620
 Cloud             0.15072
 Coffey            0.10531
 Cole              0.19368
 Comanche          0.04178
 Cooper            0.29945
 Cowley            0.19621
 Crawfor